In [1]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, KFold, cross_val_score, cross_validate, GridSearchCV,RandomizedSearchCV
import numpy as np
from matplotlib import pyplot as plt
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler

Random Forest Model Data Preparation



In [11]:
#load dat aset from csv source file
data_set = pd.read_csv('DataSets/RawDataWithStudentsRemoved.csv')

print(data_set.head())

#review data set structure
data_set.dtypes
data_set.shape

#remove null term GPA from data set
data_set = data_set.dropna(subset = ['TermGPA'])

   FakeIdentifier  Age  Gender PrimaryMilitaryAffiliation AcademicCareer  \
0               1   18  Female    No Military Affiliation  Undergraduate   
1               2   72    Male    No Military Affiliation            Law   
2               3   26    Male    No Military Affiliation       Graduate   
3               4   47  Female                    Veteran       Graduate   
4               5   45    Male    No Military Affiliation  Undergraduate   

                          College  NumberofClassesEnrolled  TermGPA  \
0              College of Science                        4     3.75   
1   James E Rogers College of Law                        3     4.00   
2          College of Engineering                        2      NaN   
3       Coll of Ag Life & Env Sci                        2     4.00   
4  College of Information Science                        2     2.00   

   CumulativeGPA  UnitsPassedincludedinGPA  UnitsPassednotincludedinGPA  \
0          3.636                         

In [14]:
print(data_set['AcademicYear'].dtype)

int64


We split the data set into a train and a test set (80/20) using the same random seed

In [ ]:
# Y target column
target_column = 'TermGPA'
test_percent = 0.20

x_columns_to_drop = ['FakeIdentifier' , 'TermGPA']

#data set dropping the target feature
x_features = data_set.drop(x_columns_to_drop, axis = 1)


#encoding categorical features
columns_to_encode = ['Gender', 'PrimaryMilitaryAffiliation', 'AcademicCareer', 'College',
                      'UAFullTimePartTime', 'AcademicLevelEndofTerm', 'FirstGenerationFlag' ]


y_target = data_set[target_column]


#split the data into a 80/20 train test split using the same random seed for consistency
x_train, x_test, y_train, y_test = train_test_split(x_features, y_target, test_size = test_percent, random_state = 698) 

#encode categorical features, and drop one level to avoid multicolinearity
x_train_encoded = pd.get_dummies(x_train, columns=columns_to_encode, drop_first=True, dtype = int)
x_test_encoded = pd.get_dummies(x_test, columns=columns_to_encode, drop_first=True, dtype = int)


In [ ]:
#making the RF Regressor Model

metric = neg_mean_squared_error
metric_name = "Mean Squared Error"
rfr_depth = 15
rfr_max_features = None # None, sqrt or log2
rfr_min_sample_split = 4
rfr_criterion = 'squared_error' #criterion{“squared_error”, “absolute_error”, “friedman_mse”, “poisson”}, default=”squared_error”
rfr_estimators = 100

model = RandomForestRegressor(n_estimators = rfr_estimators,
                               criterion = rfr_criterion,
                                 max_features = rfr_max_features,
                                    min_samples_split=rfr_min_sample_split,
                                      max_depth = rfr_depth,
                                          random_state=698)



In [ ]:
#Adding kFold Cross validation--------------------------
kfold = KFold(n_splits=5, shuffle=True, random_state=698)


#for kfold accuracy scores
cv_scores = cross_val_score(model, x_train_encoded, y_train, cv=kfold, scoring='roc_auc')

#review KFold Scores in the output (5-fold for our model)
print("Cross-validation scores:", cv_scores)

# Print the mean cross-validation score
print("Mean cross-validation score:", cv_scores.mean())

In [ ]:
#fit the model
model.fit(x_train_encoded, y_train)

# Predict using test data
y_pred = model.predict(x_test_encoded) 

    # compare prediction to actual dev/test data
score = metric(y_test, y_pred)
print(f"{metric_name} on test set: {score:.4f}")

In [ ]:
# scale the termGPA for training

#scaler = StandardScaler()
#y_train_reshape = y_train.values.reshape(-1, 1)

#y_train_zscore = scaler.fit_transform(y_train_reshape)

#y_test_zscore = 